In [25]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
NBA_36_Stats = pd.read_excel('Stats/Per_36_Stats.xlsx')
NBA_Player_Salary= pd.read_excel('Stats/Player_Salary.xlsx')


# Create an SQLite in-memory database
conn = sqlite3.connect(':memory:')

# Load DataFrames into the database as tables
NBA_36_Stats.to_sql('Stats', conn, index=False, if_exists='replace')
NBA_Player_Salary.to_sql('Salary', conn, index=False, if_exists='replace')

sql_query = """
SELECT *
FROM Stats
JOIN Salary ON Stats.Player = Salary.Player

"""


# Execute the query and load results into a DataFrame
Player_Numbers_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Stats/All_Player_Numbers.csv"
Player_Numbers_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the file to the user
Player_Numbers_df


,Player,Team,Pos,Age,G,GS,MP,FG,FGA,FG%,...,Rk,Player,Tm,Salary (2025-26),Salary (2026-27),Salary (2027-28),Salary (2028-29),Salary (2029-30),Salary (2030-31),Salary (Guaranteed)
0,Bam Adebayo,MIA,C,27,78,78,2674,7.3,15.0,0.485,...,40,Bam Adebayo,MIA,37096620,48713700,52610796,56507892,None,None,138421116
1,Ochai Agbaji,TOR,SG,24,64,45,1739,5.5,11.1,0.498,...,200,Ochai Agbaji,TOR,6383525,None,None,None,None,None,6383525
2,Grayson Allen,PHO,SG,29,64,7,1544,5.1,11.4,0.448,...,101,Grayson Allen,PHO,16875000,18125000,19375000,None,None,None,35000000
3,Jarrett Allen,CLE,C,26,82,82,2296,7.1,10.0,0.706,...,85,Jarrett Allen,CLE,20000000,28000000,30240000,32480000,None,None,110720000
4,Jose Alvarado,NOP,PG,26,56,23,1366,5.4,13.7,0.392,...,237,Jose Alvarado,NOP,4500000,4500000,None,None,None,None,4500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Zion Williamson,NOP,PF,24,30,30,857,12.1,21.3,0.567,...,29,Zion Williamson,NOP,39446090,42166510,44886930,None,None,None,126499530
320,Jalen Wilson,BRK,PF,24,79,22,2031,4.4,11.0,0.397,...,326,Jalen Wilson,BRK,2221677,None,None,None,None,None,2221677
321,James Wiseman,IND,C,23,1,0,5,14.4,28.8,0.500,...,295,James Wiseman,IND,2667947,None,None,None,None,None,1000000
322,Trae Young,ATL,PG,26,76,76,2739,7.4,18.1,0.411,...,23,Trae Young,ATL,45999660,48967380,None,None,None,None,45999660


In [29]:
import unicodedata

def normalize_name(s):
    if s is None:
        return s
    # strip accents, lower, strip spaces, collapse inner spaces
    s = unicodedata.normalize('NFKD', str(s)).encode('ascii', 'ignore').decode('ascii')
    s = ' '.join(s.strip().split()).lower()
    return s

# 1) If you want to keep only the first occurrence of duplicate columns:
Player_Numbers_df = Player_Numbers_df.loc[:, ~Player_Numbers_df.columns.duplicated()]

# 2) Identify the intended player column explicitly (first column named "Player")
player_cols = [c for c in Player_Numbers_df.columns if c == "Player"]
if not player_cols:
    raise KeyError("No 'Player' column found.")
player_col = player_cols[0]  # use the first

# 3) Normalize player names in both DF and your list
Player_Numbers_df["_player_norm"] = Player_Numbers_df[player_col].map(normalize_name)

traded_players = traded_players = [
    # --- From original trade list ---
    "Reggie Luis Jr.", "Georges Niang", "Jonas Valanciunas", "Dario Saric",
    "Kelly Olynyk", "Malaki Branham", "Blake Wesley", "Cameron Johnson",
    "Michael Porter Jr.", "Dennis Schröder", "John Collins", "Norman Powell",
    "Kevin Love", "Kyle Anderson", "Duncan Robinson", "Simone Fontecchio",
    "Kristaps Porzingis", "Terance Mann", "Drake Powell", "Anfernee Simons",
    "Jrue Holiday", "Will Richard", "Jahmai Mashack", "Justinian Jessup",
    "Alex Toohey", "Clint Capela", "Kevin Durant", "Adou Thiero",
    "Rocco Zikarsky", "Koby Brea", "Rasheer Fleming", "Daeqwon Plowden",
    "Jalen Green", "Dillon Brooks", "Khaman Maluach", "Mojave King",
    "C.J. McCollum", "Cam Whitmore", "Jordan Poole", "Saddiq Bey",
    "Micah Peavy", "Jay Huff", "Vasilije Micic", "Pat Connaughton",
    "Nickeil Alexander-Walker", "Isaac Okoro", "Lonzo Ball", "Kam Jones",
    "Lachlan Olbrich", "Mark Williams", "Liam McNeeley", "Jusuf Nurkic",
    "Collin Sexton", "Colby Jones", "Dillon Jones", "Kobe Sanders",
    "Mohamed Diawara", "Luka Mitrovic", "Amari Williams", "Max Shulga",
    "Noah Penda", "Nique Clifford", "Asa Newell", "Derik Queen",
    "Walter Clayton Jr.", "Will Riley", "Cedric Coward", "Hansen Yang",
    "Kentavious Caldwell-Pope", "Cole Anthony", "Desmond Bane",
    
    # --- From your second list ---
    "Kyrie Irving", "Myles Turner", "Fred VanVleet", "Santi Aldama",
    "Bobby Portis", "Caris LeVert", "Dorian Finney-Smith", "Davion Mitchell",
    "Luke Kennard", "Luke Kornet", "Sam Merrill", "Ty Jerome",
    "Brook Lopez", "Kevon Looney", "Tre Jones", "Tre Mann",
    "Tyus Jones", "Isaiah Jackson", "Jake LaRavia", "Ziaire Williams",
    "Day'Ron Sharpe", "D'Angelo Russell", "Nicolas Batum", "Guerschon Yabusele",
    "Paul Reed", "Kevin Porter Jr.", "Moritz Wagner", "Ryan Rollins",
    "Gary Trent Jr.", "Gary Harris", "Doug McDermott", "Spencer Dinwiddie",
    "Kyle Lowry", "Joe Ingles", "Garrett Temple", "Mason Plumlee",
    "Eric Gordon", "Chris Paul", "Jeff Green", "Tim Hardaway Jr.",
    "Larry Nance Jr.", "Taurean Prince", "Jaxson Hayes", "Dante Exum",
    "Chris Boucher", "Aaron Holiday", "Bruce Brown Jr.", "Marvin Bagley III",
    "Jordan McLaughlin", "Jae’Sean Tate", "Sandro Mamukelashvili", "Trendon Watford",
    "Luka Garza", "Jericho Sims", "Cam Spencer", "Josh Minott",
    "Jared Butler", "Lindy Waters III", "Collin Gillespie", "Justin Edwards",
    "Tyson Etienne", "Jacob Toppin", "Branden Carlson", "Emanuel Miller",
    "Yuki Kawamura", "Christian Koloko", "Spencer Jones", "Harrison Ingram",
    "Trentyn Flowers", "Riley Minix", "Keaton Wallace", "Jamal Cain",
    "Bryce McGowens", "P.J. Hall", "Jesse Edwards", "Daniss Jenkins",
    "Drew Peterson", "Trey Alexander", "Isaiah Crawford", "Tristan Vukcevic",
    "Jabari Walker", "Bradley Beal", "Deandre Ayton"
]


traded_players_norm = set(normalize_name(x) for x in traded_players)

# 4) Filter out traded players using the normalized column
Player_Numbers_df = Player_Numbers_df[~Player_Numbers_df["_player_norm"].isin(traded_players_norm)].drop(columns=["_player_norm"])
Player_Numbers_df

,Player,Team,Pos,Age,G,GS,MP,FG,FGA,FG%,...,PTS,Rk,Tm,Salary (2025-26),Salary (2026-27),Salary (2027-28),Salary (2028-29),Salary (2029-30),Salary (2030-31),Salary (Guaranteed)
0,Bam Adebayo,MIA,C,27,78,78,2674,7.3,15.0,0.485,...,19.0,40,MIA,37096620,48713700,52610796,56507892,None,None,138421116
1,Ochai Agbaji,TOR,SG,24,64,45,1739,5.5,11.1,0.498,...,13.8,200,TOR,6383525,None,None,None,None,None,6383525
2,Grayson Allen,PHO,SG,29,64,7,1544,5.1,11.4,0.448,...,15.9,101,PHO,16875000,18125000,19375000,None,None,None,35000000
3,Jarrett Allen,CLE,C,26,82,82,2296,7.1,10.0,0.706,...,17.3,85,CLE,20000000,28000000,30240000,32480000,None,None,110720000
4,Jose Alvarado,NOP,PG,26,56,23,1366,5.4,13.7,0.392,...,15.3,237,NOP,4500000,4500000,None,None,None,None,4500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Zion Williamson,NOP,PF,24,30,30,857,12.1,21.3,0.567,...,31.0,29,NOP,39446090,42166510,44886930,None,None,None,126499530
227,Jalen Wilson,BRK,PF,24,79,22,2031,4.4,11.0,0.397,...,13.3,326,BRK,2221677,None,None,None,None,None,2221677
228,James Wiseman,IND,C,23,1,0,5,14.4,28.8,0.500,...,43.2,295,IND,2667947,None,None,None,None,None,1000000
229,Trae Young,ATL,PG,26,76,76,2739,7.4,18.1,0.411,...,24.2,23,ATL,45999660,48967380,None,None,None,None,45999660


In [30]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames


# Create an SQLite in-memory database
conn = sqlite3.connect(':memory:')

# Load DataFrames into the database as tables
Player_Numbers_df.to_sql('Stats', conn, index=False, if_exists='replace')

sql_query = """
SELECT *
FROM Stats
WHERE Stats.Age BETWEEN 21 AND 31


"""


# Execute the query and load results into a DataFrame
Player_Numbers_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Stats/All_Player_Numbers.csv"
Player_Numbers_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the file to the user
Player_Numbers_df

,Player,Team,Pos,Age,G,GS,MP,FG,FGA,FG%,...,PTS,Rk,Tm,Salary (2025-26),Salary (2026-27),Salary (2027-28),Salary (2028-29),Salary (2029-30),Salary (2030-31),Salary (Guaranteed)
0,Bam Adebayo,MIA,C,27,78,78,2674,7.3,15.0,0.485,...,19.0,40,MIA,37096620,48713700,52610796,56507892,None,None,138421116
1,Ochai Agbaji,TOR,SG,24,64,45,1739,5.5,11.1,0.498,...,13.8,200,TOR,6383525,None,None,None,None,None,6383525
2,Grayson Allen,PHO,SG,29,64,7,1544,5.1,11.4,0.448,...,15.9,101,PHO,16875000,18125000,19375000,None,None,None,35000000
3,Jarrett Allen,CLE,C,26,82,82,2296,7.1,10.0,0.706,...,17.3,85,CLE,20000000,28000000,30240000,32480000,None,None,110720000
4,Jose Alvarado,NOP,PG,26,56,23,1366,5.4,13.7,0.392,...,15.3,237,NOP,4500000,4500000,None,None,None,None,4500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Zion Williamson,NOP,PF,24,30,30,857,12.1,21.3,0.567,...,31.0,29,NOP,39446090,42166510,44886930,None,None,None,126499530
226,Jalen Wilson,BRK,PF,24,79,22,2031,4.4,11.0,0.397,...,13.3,326,BRK,2221677,None,None,None,None,None,2221677
227,James Wiseman,IND,C,23,1,0,5,14.4,28.8,0.500,...,43.2,295,IND,2667947,None,None,None,None,None,1000000
228,Trae Young,ATL,PG,26,76,76,2739,7.4,18.1,0.411,...,24.2,23,ATL,45999660,48967380,None,None,None,None,45999660


In [7]:
Team_Salary_and_Finance= pd.read_excel('Stats/Team_Finance.xlsx')
Team_Salary_and_Finance

,Team,Rk,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,...,Total Tax Payroll + Bill,Active,Dead Estimated,PlayersActive,Avg AgeTeam,Total Cap Allocations,Cap Space All,Active.1,Active Top 3,Dead Cap
0,ATL,1,E,SE,40,42,0.488,-1.13,114.74,115.74,...,182383921,182383921,-,13,24.2,193262444,-38615444,182383921,106731367,-
1,BKN,2,E,A,61,21,0.744,9.11,120.83,111.32,...,114423891,114323891,100000,14,23.2,134733990,19913010,114323891,79185322,100000
2,BOS,3,E,A,26,56,0.317,-7.11,108.62,115.83,...,228082164,197685215,-,14,25.3,222376254,-67729254,197685215,135368714,-
3,CHA,4,E,SE,19,63,0.232,-9.11,107.37,116.65,...,176191763,176191763,-,18,25.3,182832805,-28185805,176191763,81933760,-
4,CHI,5,E,C,39,43,0.476,-1.56,114.28,115.77,...,149275854,149275854,-,16,25.0,180973997,-26326997,149275854,57561977,-
5,CLE,6,E,C,64,18,0.780,9.54,121.88,112.34,...,375283424,225863214,424672,13,26.7,244410298,-89763298,225863214,132234290,424672
6,DAL,7,W,SW,39,43,0.476,-1.20,114.90,116.05,...,235419773,202248038,2208856,17,26.3,226405076,-71758076,202248038,107359119,2208856
7,DEN,8,W,NW,50,32,0.610,3.89,119.97,116.20,...,189503236,188297059,-,14,26.5,206419471,-51772471,188297059,124460081,-
8,DET,9,E,C,44,38,0.537,1.90,115.05,113.12,...,164440386,164440386,-,13,25.0,171640386,-16993386,164440386,89862938,-
9,GSW,10,W,P,48,34,0.585,3.30,115.09,111.81,...,170506402,170506402,-,9,28.8,248481703,-93834703,170506402,139626124,-
